In [1]:
# Dependencies
import os
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd
import time

In [2]:
# Initialize Splinter for Windows Users
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


In [ ]:
# Initialize Splinter for Mac Users
#!which chromedriver
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [ ]:
# URL of page to be scraped
NASA_News_url = 'https://mars.nasa.gov/news/'
browser.visit(NASA_News_url)


In [ ]:
# Create Soup Object from the NASA Mars News URL
html = browser.html
NASA_Mars_News_soup = bs(html, 'html.parser')


# Confirm successful creationg of soup object.
#print(NASA_Mars_News.prettify())

In [ ]:
news_title = NASA_Mars_News_soup.find('div', class_ = 'content_title').text
print(news_title)


In [ ]:
news_p = NASA_Mars_News_soup.find('div', class_ = 'article_teaser_body').text
print(news_p)
#This is not consistently running correct

### JPL Mars Space Images - Featured Image

In [ ]:
# URL of page to visit
JPL_Mars_Images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(JPL_Mars_Images_url)
time.sleep(5)

In [ ]:
# Click on first button
browser.click_link_by_id('full_image')
time.sleep(5)

In [ ]:
# Click again
browser.click_link_by_partial_text('more info')
time.sleep(5)

In [ ]:
# Create Soup Object from the JPL Mars Space Images URL
html = browser.html
JPL_Mars_Images_soup = bs(html, 'html.parser')

# Confirm successful creationg of soup object.
#print(JPL_Mars_Images_soup.prettify())

In [ ]:
# find the relative image url
relative_img_url = JPL_Mars_Images_soup.select_one('figure.lede a img')
relative_img_url

#I found that the execution of this code is unreliable unless I put in some delays.  Maybe I am wrong.
time.sleep(10)

In [ ]:
figure = JPL_Mars_Images_soup.find('figure')
relative_mars_image_url = figure.find('a')['href']
mars_image_url = f'https://www.jpl.nasa.gov{relative_mars_image_url}'
mars_image_url

### Mars Weather

In [24]:
# URL of page to be scraped
Mars_Weather_Twitter = 'https://twitter.com/marswxreport?lang=en'
browser.visit(Mars_Weather_Twitter)

In [25]:
# Create Soup Object from the JPL Mars Weather Twitter
html=browser.html
Mars_Weather_soup= bs(html, 'html.parser')

# Confirm successful creationg of soup object.
#print(Mars_Weather_soup.prettify())

In [26]:
# Get the weather info by scraping
mars_weather = Mars_Weather_soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text



In [27]:
# Confirm success.
mars_weather

'InSight sol 379 (2019-12-20) low -96.3ºC (-141.4ºF) high -20.0ºC (-3.9ºF)\nwinds from the SSE at 5.3 m/s (11.8 mph) gusting to 20.7 m/s (46.2 mph)\npressure at 6.50 hPapic.twitter.com/O9t6gBDfHW'

### Mars Facts

In [28]:
#Read Mars Facts into dataframe --- returns a list of DataFrame objects
Mars_Facts_df = pd.read_html('https://space-facts.com/mars/')[0]

#Set Column Names and set index to description column
Mars_Facts_df.columns=['description', 'value']
Mars_Facts_df.set_index('description', inplace=True)

#View dataframe
Mars_Facts_df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [29]:
#Convert dataframe to html table
Mars_Facts_df.to_html('Mars_Facts_table.html')

### Mars Hemispheres

In [3]:
# URL of page to be scraped
Mars_Hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(Mars_Hemisphere_url)

In [4]:
# Create Soup Object from the Mars Hemisphere Web Page
html=browser.html
Mars_Hemisphere_soup= bs(html, 'html.parser')

# Confirm successful creationg of soup object.
#print(Mars_Hemisphere_soup.prettify())

In [5]:

# Retreive all items that contain mars hemispheres information
hemisphere_items = Mars_Hemisphere_soup.find_all('div', class_='item')

# Create an empty list to store scraped hemisphere urls 
hemisphere_image_urls = []

# Variable for the main web site 
hemispheres_main_url = 'https://astrogeology.usgs.gov'



In [13]:
# Loop through the hemisphere items
for i in hemisphere_items: 
    # Store hemisphere title
    hemisphere_title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the complete image url 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # Create HTML Object of individual hemisphere image information 
    partial_img_html = browser.html
    
    # Create soup object using partial image url 
    partial_img_soup = bs( partial_img_html, 'html.parser')
    
    # Create complete image source url
    complete_img_url = hemispheres_main_url + partial_img_soup.find('img', class_='wide-image')['src']
    
    # Add title/image url dictionaries to hemisphere_image_urls list
    hemisphere_image_urls.append({"title" : hemisphere_title, "img_url" : complete_img_url})

In [14]:
#Check to see that the list of dictionaries look like
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}, {'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_

In [18]:
#NOT NEEDED
#Create dataframe from the hemisphere image url list of dictionaries
#hemisphere_image_url_df = pd.DataFrame(hemisphere_image_urls)
#hemisphere_image_url_df

In [17]:
#NOT NEEDED
#Convert hemisphere image url dataframe to html
#hemisphere_image_url_df.to_html('Mars_Hemisphere_image_urls.html')